In [1]:
import pandas as pd
import numpy as np
import re

%matplotlib inline

from matplotlib import pyplot as plt
plt.style.use('ggplot')

import seaborn as sns

In [2]:
full_data = pd.read_csv("roto_scrape.csv", sep = ';', index_col=0)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (4,5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Calculate Player Value
**In this notebook I try to assign value to players considering points scored vs salary. Since the individual player points each week is highly volatile and hard to predict, I assume the best prediction for points is the average of the previous weeks point totals. My metric for value is 'Cost per Point', using the assumption that the player will score his average each week. To do this calculation I create columns for indiviudal players cumulative point totals for the season and a running count of games played. I could not simpy use the week column because if a player is injured or sits out a week I did not want the 0 values to scew my data. Once I calculate the average points, I use the latest salary data point to calculate expected cost per point for that player.**

In [3]:
data_2021 = full_data[full_data['Year'] == 2021]
data_2021

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary
416063,1.0,2021.0,1537.0,"Murray, Kyler",QB,ari,a,ten,34.56,8400.0,34.56,7600.0,34.56,34.0
416064,1.0,2021.0,1523.0,"Mahomes II, Patrick",QB,kan,h,cle,33.28,8800.0,36.28,8100.0,33.28,40.0
416065,1.0,2021.0,1490.0,"Goff, Jared",QB,det,h,sfo,29.92,6500.0,32.92,5100.0,29.92,20.0
416066,1.0,2021.0,1465.0,"Winston, Jameis",QB,nor,h,gnb,29.62,6700.0,29.62,5200.0,29.62,24.0
416067,1.0,2021.0,1131.0,"Brady, Tom",QB,tam,h,dal,29.16,7800.0,32.16,6700.0,29.16,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418210,5.0,2021.0,7020.0,New York G,Def,nyg,a,dal,2.00,3000.0,2.00,2600.0,2.00,10.0
418211,5.0,2021.0,7008.0,Cleveland,Def,cle,a,lac,0.00,4100.0,0.00,2700.0,0.00,16.0
418212,5.0,2021.0,7015.0,Kansas City,Def,kan,h,buf,-1.00,3300.0,-1.00,3000.0,-1.00,11.0
418213,5.0,2021.0,7016.0,Miami,Def,mia,a,tam,-2.00,3600.0,-2.00,2400.0,-2.00,12.0


In [22]:
fd_cumpoints = data_2021.groupby('Name').apply(lambda grp: np.cumsum(grp.sort_values('Week')[['FD points']])).reset_index()
fd_cumpoints = fd_cumpoints.drop('Name', axis=1).rename({'FD points': 'FD_cum_points'}, axis=1)

In [23]:
fd_cumpoints.columns

Index(['level_1', 'FD_cum_points'], dtype='object')

In [24]:
FD_cumulative = pd.merge(data_2021, fd_cumpoints, left_index=True, right_on='level_1')

In [25]:
FD_cumulative[FD_cumulative.Name == 'Wilson, Russell']

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points
2112,1.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,ind,27.06,7800.0,27.06,7000.0,27.06,32.0,416070,27.06
2113,2.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,ten,23.32,8500.0,26.32,7500.0,23.32,35.0,416501,50.38
2114,3.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,min,16.62,8400.0,16.62,7600.0,16.62,36.0,416948,67.00
2115,4.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,sfo,22.56,7700.0,22.56,7100.0,22.56,34.0,417367,89.56
2116,5.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,lar,10.08,7800.0,10.08,7000.0,10.08,33.0,417815,99.64


In [26]:
FD_cumulative['Bool'] = [1 if x > 0 else 0 for x in FD_cumulative['FD points']]

In [27]:
FD_cumulative['Games Played'] = FD_cumulative.groupby('Name')['Bool'].cumsum()

In [28]:
FD_cumulative[FD_cumulative.Name == 'Wilson, Russell']

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Games Played
2112,1.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,ind,27.06,7800.0,27.06,7000.0,27.06,32.0,416070,27.06,1,1
2113,2.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,ten,23.32,8500.0,26.32,7500.0,23.32,35.0,416501,50.38,1,2
2114,3.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,min,16.62,8400.0,16.62,7600.0,16.62,36.0,416948,67.00,1,3
2115,4.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,sfo,22.56,7700.0,22.56,7100.0,22.56,34.0,417367,89.56,1,4
2116,5.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,lar,10.08,7800.0,10.08,7000.0,10.08,33.0,417815,99.64,1,5


In [29]:
FD_cumulative['Average FD Points'] = FD_cumulative['FD_cum_points'] / FD_cumulative['Games Played']

In [30]:
FD_cumulative[FD_cumulative.Name == 'Wilson, Russell']

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Games Played,Average FD Points
2112,1.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,ind,27.06,7800.0,27.06,7000.0,27.06,32.0,416070,27.06,1,1,27.060000
2113,2.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,ten,23.32,8500.0,26.32,7500.0,23.32,35.0,416501,50.38,1,2,25.190000
2114,3.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,min,16.62,8400.0,16.62,7600.0,16.62,36.0,416948,67.00,1,3,22.333333
2115,4.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,sfo,22.56,7700.0,22.56,7100.0,22.56,34.0,417367,89.56,1,4,22.390000
2116,5.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,lar,10.08,7800.0,10.08,7000.0,10.08,33.0,417815,99.64,1,5,19.928000


In [31]:
FD_cumulative['Price_per_Point'] = FD_cumulative['FD salary'] / FD_cumulative['Average FD Points']

In [32]:
FD_cumulative[FD_cumulative.Name == 'Wilson, Russell']

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Games Played,Average FD Points,Price_per_Point
2112,1.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,ind,27.06,7800.0,27.06,7000.0,27.06,32.0,416070,27.06,1,1,27.060000,288.248337
2113,2.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,ten,23.32,8500.0,26.32,7500.0,23.32,35.0,416501,50.38,1,2,25.190000,337.435490
2114,3.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,min,16.62,8400.0,16.62,7600.0,16.62,36.0,416948,67.00,1,3,22.333333,376.119403
2115,4.0,2021.0,1412.0,"Wilson, Russell",QB,sea,a,sfo,22.56,7700.0,22.56,7100.0,22.56,34.0,417367,89.56,1,4,22.390000,343.903528
2116,5.0,2021.0,1412.0,"Wilson, Russell",QB,sea,h,lar,10.08,7800.0,10.08,7000.0,10.08,33.0,417815,99.64,1,5,19.928000,391.409073


# Once the value metric is assigned. I sort players at each position by 'highest value'. These will be players to target for future lineups.

In [33]:
wr_value = FD_cumulative.loc[FD_cumulative['Pos'] == 'WR']

In [35]:
wr_value[(wr_value['Week'] == 5) & (wr_value['Games Played'] > 3)].sort_values('Price_per_Point', ascending = True)[:25]

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Games Played,Average FD Points,Price_per_Point
253,5.0,2021.0,5730.0,"Brown, Marquise",WR,bal,h,ind,29.0,6800.0,36.5,5800.0,29.0,21.0,417938,89.60,1,5,17.920,379.464286
2095,5.0,2021.0,5594.0,"Williams, Mike",WR,lac,h,cle,32.5,7700.0,39.5,7200.0,32.5,25.0,417936,100.60,1,5,20.120,382.703777
1677,5.0,2021.0,5726.0,"Samuel, Deebo",WR,sfo,a,ari,14.6,7700.0,16.1,7100.0,14.6,24.0,417954,94.50,1,5,18.900,407.407407
1194,5.0,2021.0,5552.0,"Kupp, Cooper",WR,lar,a,sea,12.7,8400.0,16.2,7900.0,12.7,37.0,417959,100.30,1,5,20.060,418.743769
351,5.0,2021.0,5924.0,"Chase, Ja'Marr",WR,cin,h,gnb,24.9,7300.0,30.9,5800.0,24.9,20.0,417941,86.90,1,5,17.380,420.023015
234,5.0,2021.0,3474.0,"Brown, Antonio",WR,tam,h,mia,27.9,6500.0,34.4,5200.0,27.9,19.0,417939,61.10,1,4,15.275,425.531915
890,5.0,2021.0,5485.0,"Hill, Tyreek",WR,kan,h,buf,11.3,8500.0,14.8,8900.0,11.3,32.0,417963,96.60,1,5,19.320,439.958592
601,5.0,2021.0,5253.0,"Evans, Mike",WR,tam,h,mia,26.3,7000.0,32.3,6800.0,26.3,24.0,417940,77.80,1,5,15.560,449.871465
9,5.0,2021.0,5263.0,"Adams, Davante",WR,gnb,a,cin,32.1,8200.0,40.6,8200.0,32.1,33.0,417937,90.90,1,5,18.180,451.045105
1319,5.0,2021.0,5725.0,"Metcalf, D.K.",WR,sea,h,lar,24.3,7400.0,26.8,7000.0,24.3,25.0,417943,80.80,1,5,16.160,457.920792


In [37]:
wr_value_sorted = wr_value[(wr_value['Week'] == 5) & (wr_value['Games Played'] > 3)].sort_values('Price_per_Point', ascending = True)

In [39]:
wr_value_sorted[['Name', "Pos", 'FD salary', 'FD_cum_points', 'Games Played', 'Average FD Points', 'Price_per_Point']][:10]

,Name,Pos,FD salary,FD_cum_points,Games Played,Average FD Points,Price_per_Point
253,"Brown, Marquise",WR,6800.0,89.6,5,17.920,379.464286
2095,"Williams, Mike",WR,7700.0,100.6,5,20.120,382.703777
1677,"Samuel, Deebo",WR,7700.0,94.5,5,18.900,407.407407
1194,"Kupp, Cooper",WR,8400.0,100.3,5,20.060,418.743769
351,"Chase, Ja'Marr",WR,7300.0,86.9,5,17.380,420.023015
234,"Brown, Antonio",WR,6500.0,61.1,4,15.275,425.531915
890,"Hill, Tyreek",WR,8500.0,96.6,5,19.320,439.958592
601,"Evans, Mike",WR,7000.0,77.8,5,15.560,449.871465
9,"Adams, Davante",WR,8200.0,90.9,5,18.180,451.045105
1319,"Metcalf, D.K.",WR,7400.0,80.8,5,16.160,457.920792


In [54]:
qb_value = FD_cumulative.loc[FD_cumulative['Pos'] == 'QB']

In [69]:
qb_value[(qb_value['Week'] == 5) & (qb_value['Week Count'] > 3)].sort_values('Price_per_Point', ascending = True)[:25]

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Week Count,Average FD Points,Price_per_Point
847,5.0,2021.0,1552.0,"Herbert, Justin",QB,lac,h,cle,42.82,7700.0,45.82,6800.0,42.82,34.0,417789,126.04,1,5,25.208,305.458585
1014,5.0,2021.0,1527.0,"Jackson, Lamar",QB,bal,h,ind,41.88,8200.0,45.88,7600.0,41.88,36.0,417790,133.86,1,5,26.772,306.290154
204,5.0,2021.0,1131.0,"Brady, Tom",QB,tam,h,mia,37.74,8400.0,40.74,7400.0,37.74,38.0,417791,136.28,1,5,27.256,308.189023
1256,5.0,2021.0,1523.0,"Mahomes II, Patrick",QB,kan,h,buf,20.98,8700.0,21.98,8200.0,20.98,37.0,417801,136.90,1,5,27.380,317.750183
977,5.0,2021.0,1554.0,"Hurts, Jalen",QB,phi,a,car,23.92,8100.0,23.92,7000.0,23.92,28.0,417795,125.20,1,5,25.040,323.482428
1415,5.0,2021.0,1537.0,"Murray, Kyler",QB,ari,h,sfo,13.66,8500.0,13.66,8000.0,13.66,41.0,417811,125.48,1,5,25.096,338.699394
45,5.0,2021.0,1529.0,"Allen, Josh",QB,buf,a,kan,36.50,8800.0,39.50,8100.0,36.50,41.0,417792,129.60,1,5,25.920,339.506173
470,5.0,2021.0,1524.0,"Darnold, Sam",QB,car,h,phi,9.08,7600.0,9.08,6600.0,9.08,29.0,417817,106.84,1,5,21.368,355.672033
1847,5.0,2021.0,1340.0,"Stafford, Matthew",QB,lar,a,sea,17.50,7900.0,20.50,6500.0,17.50,34.0,417805,109.88,1,5,21.976,359.483072
1102,5.0,2021.0,1539.0,"Jones, Daniel",QB,nyg,a,dal,4.82,7400.0,4.82,6000.0,4.82,25.0,417822,99.98,1,5,19.996,370.074015


In [55]:
rb_value = FD_cumulative.loc[FD_cumulative['Pos'] == 'RB']

In [70]:
rb_value[(rb_value['Week'] == 5) & (rb_value['Week Count'] > 3)].sort_values('Price_per_Point', ascending = True)[:25]

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Week Count,Average FD Points,Price_per_Point
567,5.0,2021.0,5575.0,"Ekeler, Austin",RB,lac,h,cle,30.4,7700.0,33.9,7600.0,30.4,28.0,417829,107.8,1,5,21.560,357.142857
837,5.0,2021.0,5445.0,"Henry, Derrick",RB,ten,a,jac,31.0,10400.0,34.0,9000.0,31.0,42.0,417827,127.5,1,5,25.500,407.843137
967,5.0,2021.0,5560.0,"Hunt, Kareem",RB,cle,a,lac,23.4,7000.0,25.9,5800.0,23.4,21.0,417834,84.9,1,5,16.980,412.249706
832,5.0,2021.0,5716.0,"Henderson, Darrell",RB,lar,a,sea,16.4,6400.0,16.9,5900.0,16.4,23.0,417842,61.4,1,4,15.350,416.938111
576,5.0,2021.0,5444.0,"Elliott, Ezekiel",RB,dal,h,nyg,24.2,8000.0,28.2,7000.0,24.2,32.0,417832,91.2,1,5,18.240,438.596491
814,5.0,2021.0,5904.0,"Harris, Najee",RB,pit,h,den,21.2,7300.0,25.2,6900.0,21.2,21.0,417836,82.5,1,5,16.500,442.424242
1905,5.0,2021.0,5820.0,"Taylor, Jonathan",RB,ind,a,bal,30.4,6900.0,34.9,6300.0,30.4,25.0,417828,77.4,1,5,15.480,445.736434
1880,5.0,2021.0,5822.0,"Swift, D'Andre",RB,det,a,min,19.4,6900.0,22.4,6100.0,19.4,22.0,417839,76.7,1,5,15.340,449.804433
361,5.0,2021.0,5622.0,"Chubb, Nick",RB,cle,a,lac,23.5,7500.0,27.0,6700.0,23.5,28.0,417833,80.3,1,5,16.060,466.998755
1404,5.0,2021.0,5824.0,"Moss, Zack",RB,buf,a,kan,10.7,6300.0,12.2,5500.0,10.7,19.0,417853,53.8,1,4,13.450,468.401487


In [56]:
te_value = FD_cumulative.loc[FD_cumulative['Pos'] == 'TE']

In [71]:
te_value[(te_value['Week'] == 5) & (te_value['Week Count'] > 3)].sort_values('Price_per_Point', ascending = True)[:25]

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Week Count,Average FD Points,Price_per_Point
67,5.0,2021.0,4721.0,"Andrews, Mark",TE,bal,h,ind,36.2,6400.0,44.7,5400.0,36.2,20.0,418094,70.5,1,5,14.100,453.900709
1180,5.0,2021.0,4750.0,"Knox, Dawson",TE,buf,a,kan,19.2,6000.0,23.7,4300.0,19.2,19.0,418097,65.1,1,5,13.020,460.829493
1450,5.0,2021.0,4702.0,"Njoku, David",TE,cle,a,lac,24.4,4700.0,30.9,2900.0,24.4,10.0,418095,39.0,1,4,9.750,482.051282
1711,5.0,2021.0,4742.0,"Schultz, Dalton",TE,dal,h,nyg,10.9,6200.0,13.9,4400.0,10.9,19.0,418101,59.0,1,5,11.800,525.423729
1155,5.0,2021.0,4582.0,"Kelce, Travis",TE,kan,h,buf,14.7,8200.0,17.7,7600.0,14.7,31.0,418099,75.9,1,5,15.180,540.184453
697,5.0,2021.0,4719.0,"Gesicki, Mike",TE,mia,a,tam,6.3,5600.0,8.3,4200.0,6.3,16.0,418114,39.7,1,4,9.925,564.231738
1543,5.0,2021.0,4801.0,"Pitts, Kyle",TE,atl,h,nyj,22.4,5800.0,29.9,4700.0,22.4,16.0,418096,48.8,1,5,9.760,594.262295
842,5.0,2021.0,4675.0,"Henry, Hunter",TE,nwe,a,hou,16.5,5400.0,19.5,3700.0,16.5,16.0,418098,43.6,1,5,8.720,619.266055
2090,5.0,2021.0,4650.0,"Williams, Maxx",TE,ari,h,sfo,1.9,5200.0,2.4,3400.0,1.9,12.0,418141,33.3,1,4,8.325,624.624625
1994,5.0,2021.0,5419.0,"Waller, Darren",TE,lvr,h,chi,6.5,7400.0,8.5,7300.0,6.5,23.0,418113,57.9,1,5,11.580,639.032815


In [57]:
def_value = FD_cumulative.loc[FD_cumulative['Pos'] == 'DEF']

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,FD points,FD salary,DK points,DK salary,YH points,YH salary,level_1,FD_cum_points,Bool,Week Count,Average FD Points,Price_per_Point


*FanDuel does not include defenses in their contests*